In [3]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(69)

In [3]:
# 학습 데이터 불러오기
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Taeho/LGAimers/train.csv') # 학습용 데이터
# 예측할 데이터 불러오기
submission_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Taeho/LGAimers/submission.csv') # 테스트 데이터(제출파일의 데이터)

In [4]:
train_data['sample_weight'] = 1

# 양성 클래스의 샘플에 대해 가중치를 증가 (예: 3으로 설정)
train_data.loc[train_data['is_converted'] == 1, 'sample_weight'] = 4

print(train_data)

       bant_submit          customer_country business_unit  \
0             1.00  /Quezon City/Philippines            AS   
1             1.00        /PH-00/Philippines            AS   
2             1.00           /Kolkata /India            AS   
3             1.00        /Bhubaneswar/India            AS   
4             1.00          /Hyderabad/India            AS   
...            ...                       ...           ...   
59294         1.00           /Sląskie/Poland            AS   
59295         0.75      /Bogotá DC /Colombia            AS   
59296         0.75               /Pisco/Peru            AS   
59297         1.00  /santa cruz bolivia/Peru            AS   
59298         0.75             /paris/France      Solution   

       com_reg_ver_win_rate  customer_idx          customer_type  enterprise  \
0                  0.066667         32160           End-Customer  Enterprise   
1                  0.066667         23122           End-Customer  Enterprise   
2              

In [5]:
from autogluon.tabular import TabularPredictor
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(69)
label = 'is_converted'
predictor = TabularPredictor(label=label, eval_metric='f1',
                             sample_weight = 'sample_weight'
                             ,weight_evaluation = True).fit(
    train_data=train_data,
    presets='best_quality',
#   time_limit=3600*6,
    auto_stack=True,
    num_cpus=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240225_133651"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240225_133651/ds_sub_fit/sub_fit_ho.
Values in column 'sample_weight' used as sample weights instead of predictive features. Evaluation will report weighted metrics, so ensure same column exists in test data.
Beginning AutoGluon training ... Time limi

In [6]:
print(predictor.leaderboard(silent=True))

model_pred = predictor.predict(submission_data.drop(columns=[label]))

                      model  score_val eval_metric  pred_time_val  \
0       WeightedEnsemble_L3   0.925876          f1      65.639077   
1           CatBoost_BAG_L2   0.925162          f1      65.376840   
2       WeightedEnsemble_L2   0.908135          f1      38.459161   
3         LightGBMXT_BAG_L2   0.907789          f1      65.271729   
4           CatBoost_BAG_L1   0.906708          f1       0.413604   
5      LightGBMLarge_BAG_L2   0.904227          f1      65.309479   
6           LightGBM_BAG_L2   0.903674          f1      65.262225   
7    NeuralNetFastAI_BAG_L2   0.903318          f1      66.360581   
8            XGBoost_BAG_L2   0.902214          f1      65.471277   
9   RandomForestEntr_BAG_L2   0.888914          f1      66.946450   
10          LightGBM_BAG_L1   0.888517          f1       9.281972   
11    NeuralNetTorch_BAG_L2   0.887589          f1      65.886916   
12    ExtraTreesGini_BAG_L2   0.887351          f1      67.146015   
13    ExtraTreesEntr_BAG_L2   0.88

In [7]:
# 최종 제출 파일 생성
submission_data[label] = model_pred
final_submission_path = '/content/drive/MyDrive/Colab Notebooks/Taeho/LGAimers/submission_Raw_AG.csv'
submission_data.to_csv(final_submission_path, index=False, encoding="utf-8")

final_submission_path

'/content/drive/MyDrive/Colab Notebooks/Taeho/LGAimers/submission_Raw_AG.csv'

In [12]:
submission_data['is_converted'] = submission_data['is_converted'].astype(int)

In [14]:
converted_count = submission_data['is_converted'].value_counts()

In [15]:
converted_count

is_converted
0    4030
1    1241
Name: count, dtype: int64

In [16]:
final_submission_path = '/content/drive/MyDrive/Colab Notebooks/Taeho/LGAimers/submission_Raw_AG.csv'
submission_data.to_csv(final_submission_path, index=False, encoding="utf-8")

In [ ]:
train_data = TabularDataset(train_data)
submission_data = TabularDataset(submission_data)

label = 'is_converted'  # 레이블 이름 업데이트
eval_metric = 'f1'  # 평가 메트릭을 F1으로 설정
# time_limit = 3600 * 6  # 필요에 따라 조정

In [ ]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data,
      presets='best_quality',
#     time_limit=time_limit,
      num_gpus=1
      )

In [ ]:
print(predictor.leaderboard(silent=True))

model_pred = predictor.predict(submission_data.drop(columns=[label]))

In [ ]:
# 최종 제출 파일 생성
submission_data[label] = model_pred
final_submission_path = '/content/drive/MyDrive/Colab Notebooks/Taeho/LGAimers/submission_Raw_AG.csv'
submission_data.to_csv(final_submission_path, index=False, encoding="utf-8")

final_submission_path

In [ ]:
converted_count = submission_data['is_converted'].value_counts()

In [ ]:
converted_count